In [ ]:
# Limpiamos todas las variables
%reset -f


# Integrantes
|N° SIU| NOMBRE APELLIDO--------------| CORREO---------------------|
|-----|-------------------------------|----------------------------|
|a1904|	Marck Anthony Murillo Ronquillo| murillo.ronq.50c2@gmail.com|
|a1906|	Yandri Jair	Uchuari Gallardo| yandriuchuari@gmail.com|
|a1823|	Luis Alberto Santamaria Jimenez| santamaria.luigi@gmail.com|

# **TACO Trash -	[taco-trash-dataset](https://www.kaggle.com/datasets/kneroma/tacotrashdataset)**

### Tipo de problema : Detección
- Objetivo detección de residuos: Utilizar imágenes para identificar la presencia de residuos. Este modelo puede ser útil para planificar limpiar y preservar estos espacios.​
- Entrenar un modelo para identificar la presencia de residuos plásticos en imágenes.


### **Descripción general**
En el presente Notebook estamos realizando un testing de entrenamiento utilizando la arquitectura de yolo 11 tomando en cuenta los siguientes puntos

- Para determinar un baseline se realizó una búsqueda de otros trabajos encontrando que las arquitecturas de 2 pasos YOLOv5 han sido utilizadas para resolver este tipo de problemas obteniendo buenos resultados en la detección de basura en tiempo real
- Se realizó una primera prueba con la arquitectura YOLOv5 en Google colab donde obtuvimos un mAP (precisión media promedio) de 0.023 principalmente debido a limitaciones computacionales
- Después se probó con la arquitectura YOLOv11 dado que uno de los integrantes la había usado anteriormente obtuvimos un mejor resultado.
- Podemos comentar que entre los testing realizado consideramos mas siguientes configuraciones como las mas optimas   
    - Establece el número de épocas de entrenamiento en `epochs=50`
    -	De momento Todas las imágenes se redimensionan a 640x640 píxeles antes de incorporarse al modelo, aun estamos definiendo cual es el tamaño optimo puesto que tenemos imágenes muy grandes y objetos a detectar muy pequeños teniendo en si un mal etiquetado, estamos haciendo pruebas redimensionando a `imgsz=1542` y ver que tanto mejora
    -	Se procesarán 16 imágenes simultáneamente durante cada paso de entrenamiento antes de que se actualicen los pesos del modelo `batch=16`. Durante las pruebas con yolov5 el batch se probó con 64 y el resultado fue muy lento el entrenamiento
    - Nos queda pendiente el testing de las distintas variantes del modelo  Yolo11 (YOLOv11n -Nano, YOLOv11s - Small, YOLOv11m medium, YOLOv11l - Large , YOLOv11x - Extra Large) ver que tanto mejora la detección  



modelos pre-entrenados: [Ultralytics YOLO11](https://huggingface.co/Ultralytics/YOLO11)




In [ ]:
# Descargamos archivos y descomprimimos
!pip install gdown
# https://drive.google.com/file/d/1ORFJYbu2-O30vm609xddWSBZjvGvaYE9/view?usp=sharing
!gdown https://drive.google.com/uc?id=1ORFJYbu2-O30vm609xddWSBZjvGvaYE9 -O archive.zip


Downloading...
From (original): https://drive.google.com/uc?id=1ORFJYbu2-O30vm609xddWSBZjvGvaYE9
From (redirected): https://drive.google.com/uc?id=1ORFJYbu2-O30vm609xddWSBZjvGvaYE9&confirm=t&uuid=47df5e83-875b-4ead-9e75-c673ec1c3582
To: /home/ose/Documents/Proyecto Yandri CEIA/archive.zip
100%|██████████████████████████████████████| 2.99G/2.99G [02:29<00:00, 20.0MB/s]
unzip:  cannot find or open /content/archive.zip, /content/archive.zip.zip or /content/archive.zip.ZIP.


In [ ]:
!unzip archive.zip -d taco_trash_dataset

Archive:  archive.zip
  inflating: taco_trash_dataset/best-checkpoint-003epoch.bin  
  inflating: taco_trash_dataset/data/annotations.json  
  inflating: taco_trash_dataset/data/batch_1/000000.jpg  
  inflating: taco_trash_dataset/data/batch_1/000001.jpg  
  inflating: taco_trash_dataset/data/batch_1/000003.jpg  
  inflating: taco_trash_dataset/data/batch_1/000004.jpg  
  inflating: taco_trash_dataset/data/batch_1/000005.jpg  
  inflating: taco_trash_dataset/data/batch_1/000006.jpg  
  inflating: taco_trash_dataset/data/batch_1/000007.jpg  
  inflating: taco_trash_dataset/data/batch_1/000008.jpg  
  inflating: taco_trash_dataset/data/batch_1/000010.jpg  
  inflating: taco_trash_dataset/data/batch_1/000011.jpg  
  inflating: taco_trash_dataset/data/batch_1/000012.jpg  
  inflating: taco_trash_dataset/data/batch_1/000013.jpg  
  inflating: taco_trash_dataset/data/batch_1/000014.jpg  
  inflating: taco_trash_dataset/data/batch_1/000015.jpg  
  inflating: taco_trash_dataset/data/batch_1/00

In [ ]:
!pip install pycocotools


In [ ]:
import os
import shutil
import numpy as np
import tqdm
from pycocotools.coco import COCO
data_source = COCO(annotation_file='/home/ose/Documents/Proyecto Yandri CEIA/taco_trash_dataset/data/annotations.json')

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [ ]:
# remapping label id to 0~1

# 'Clear plastic bottle': 5  -> 0
# 'Plastic bottle cap': 7 -> 1
# 'Drink can': 12 -> 2
# 'Other plastic': 29 -> 3
# 'Plastic film': 36 -> 4
# 'Other plastic wrapper': 39 -> 5
# 'Unlabeled litter': 58 -> 6
# 'Cigarette': 59 -> 7
label_transfer = {5: 0,
                  7: 1,
                  12: 2,
                  29: 3,
                  36: 4,
                  39: 5,
                  58: 6}


In [ ]:
img_ids = data_source.getImgIds()

catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])
classes = {}
coco_labels = {}
coco_labels_inverse = {}
for c in categories:
    coco_labels[len(classes)] = c['id']
    coco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

class_num = {}

In [ ]:
!mkdir -p tmp/labels tmp/images
save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'

In [ ]:
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc='change .json file to .txt file'):
    img_info = data_source.loadImgs(img_id)[0]

    save_name = img_info['file_name'].replace('/', '_')

    file_name = save_name.split('.')[0]

    height = img_info['height']
    width = img_info['width']

    save_path = save_base_path + file_name + '.txt'
    is_exist = False
    with open(save_path, mode='w') as fp:

        annotation_id = data_source.getAnnIds(img_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0:
            fp.write('')
            continue

        annotations = data_source.loadAnns(annotation_id)
        lines = ''

        for annotation in annotations:

            label = coco_labels_inverse[annotation['category_id']]
            if label in label_transfer.keys():

                is_exist = True
                box = annotation['bbox']
                if box[2] < 1 or box[3] < 1:

                    continue

                box[0] = round((box[0] + box[2] / 2) / width, 6)
                box[1] = round((box[1] + box[3] / 2) / height, 6)
                box[2] = round(box[2] / width, 6)
                box[3] = round(box[3] / height, 6)
                label = label_transfer[label]
                if label not in class_num.keys():
                    class_num[label] = 0
                class_num[label] += 1
                lines = lines + str(label)
                for i in box:
                    lines += ' ' + str(i)
                lines += '\n'
        fp.writelines(lines)
    if is_exist:

        shutil.copy('/home/ose/Documents/Proyecto Yandri CEIA/taco_trash_dataset/data/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:

        os.remove(save_path)

change .json file to .txt file: 1500it [00:00, 1840.16it/s]


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('tmp', output="taco/", seed=1337, ratio=(.8, 0.1,0.1))





Copying files: 2034 files [00:01, 1931.51 files/s]


In [ ]:
# Entrenamiento con YOLOv11
from ultralytics import YOLO

# Cargar modelo YOLOv11 preentrenado
model = YOLO("yolo11n.pt")  # Puedes usar yolov11s.pt o yolov11m.pt según recursos

# Entrenar
model.train(data="taco_trash.yaml", epochs=50, imgsz=640, batch=16)

# Evaluar (opcional)
model.val()

Copying files: 0 files [03:53, ? files/s]


WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at '/home/ose/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.170 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24241MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=taco_trash.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, 

train: Scanning /home/ose/Documents/Proyecto Yandri CEIA/taco/train/labels... 813 images, 3 backgrounds, 0 corrupt: 100%|██████████| 813/813 [00:00<00:00, 3829.88it/s]

train: New cache created: /home/ose/Documents/Proyecto Yandri CEIA/taco/train/labels.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3666.6±2792.0 MB/s, size: 1081.6 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.50it/s]


                   all        101        224      0.337      0.321       0.29      0.216
  Clear plastic bottle         20         24      0.555      0.792      0.788      0.686
             Drink can         22         26      0.294      0.192      0.182     0.0957
    Plastic bottle cap         19         35      0.479      0.498      0.533      0.401
         Other plastic         16         24      0.123     0.0417     0.0412     0.0229
          Plastic film         35         39      0.485      0.338      0.293       0.21
 Other plastic wrapper         16         24       0.24      0.329      0.129     0.0645
      Unlabeled litter         26         52      0.186     0.0577     0.0613     0.0307
Speed: 0.3ms preprocess, 0.7ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train
Ultralytics 8.3.170 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24241MiB)
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients,

val: Scanning /home/ose/Documents/Proyecto Yandri CEIA/taco/val/labels.cache... 101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 101/101 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.63it/s]


                   all        101        224      0.337       0.32      0.292      0.219
  Clear plastic bottle         20         24      0.557      0.792      0.795      0.694
             Drink can         22         26      0.296      0.192      0.182     0.0964
    Plastic bottle cap         19         35      0.489       0.52      0.547      0.415
         Other plastic         16         24      0.126     0.0417     0.0357     0.0191
          Plastic film         35         39      0.473      0.359      0.293      0.207
 Other plastic wrapper         16         24      0.217      0.277      0.133     0.0676
      Unlabeled litter         26         52      0.201     0.0577      0.062     0.0302
Speed: 1.8ms preprocess, 9.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78a3f271dbe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [ ]:
!yolo task=detect mode=train model=yolo11n.pt data=taco_trash.yaml epochs=150 batch=64 plots=True device=0 conf=0.2 patience=30 warmup_epochs=3 save_period=10 iou=0.35

#testing  configuración
#!yolo task=detect mode=train model=bestm.pt data=taco_trash.yaml epochs=50 batch=8 plots=True device=0 conf=0.45 patience=30  warmup_epochs=15 save_period=5 iou=0.92 \
#    lr0=0.0015 \
#    lrf=0.00015 \
#    weight_decay=0.0003 \
#    optimizer=AdamW \
#    amp=True \
#    pretrained=True \
#    imgz = 1234 \


Ultralytics 8.3.170 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24241MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.6, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=taco_trash.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.35, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots=True, pose=12.0, pretrain